<a href="https://colab.research.google.com/github/fraserstark/Coursera_TensorFlow_Labs/blob/master/Introduction/Exercise_4_Coursera_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

Below is code with a link to a happy or sad dataset which contains 80 images, 40 happy and 40 sad. 
The code creates a convolutional neural network that trains to 100% accuracy on these images,  which cancels training upon hitting training accuracy of >.999


In [14]:
import tensorflow as tf
import os
import zipfile

!wget --no-check-certificate \
    "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip" \
    -O "/tmp/happy-or-sad.zip"

zip_ref = zipfile.ZipFile("/tmp/happy-or-sad.zip", 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()


--2020-09-30 20:36:27--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.215.128, 173.194.216.128, 108.177.12.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.215.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2.5M) [application/zip]
Saving to: ‘/tmp/happy-or-sad.zip’

/tmp/happy-or-sad.z 100%[===================>]   2.55M  --.-KB/s    in 0.01s   

2020-09-30 20:36:27 (184 MB/s) - ‘/tmp/happy-or-sad.zip’ saved [2670333/2670333]



There are numerous callback types at our disposal, but in this example I'll create a simple custom callback method which will stop the model training once the desired accuracy is reached, in this case, 99.9%

In [15]:
DESIRED_ACCURACY = 0.999

class myCallback(tf.keras.callbacks.Callback):
 def on_epoch_end(self, epoch, logs={}):
         if(logs.get('accuracy')>DESIRED_ACCURACY):
          print("\nReached 99.9% accuracy so cancelling training!")
          self.model.stop_training = True

callbacks = myCallback()

This code will define and compile the model.  Note: the input shape is the desired size of the image 150x150 with 3 bytes colour.

In [16]:
model = tf.keras.models.Sequential([
 
    # This is the first convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
        
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
        
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
        
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
        
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
        
    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('happy') and 1 for the other ('sad')
    tf.keras.layers.Dense(1, activation='sigmoid')
])


The model.summary() method call prints a summary of the NN.

In [17]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 148, 148, 16)      448       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 72, 72, 32)        4640      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 17, 17, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 18496)            

Here, the 'output shape' column shows how the size of the feature map evolves in each successive layer. The convolution layers reduce the size of the feature maps by a bit due to padding, and each pooling layer halves the dimensions.

Next, we'll configure the specifications for model training. We will train our model with the binary_crossentropy loss, because it's a binary classification problem and our final activation is a sigmoid. We will use the rmsprop optimizer with a learning rate of 0.001. During training, we will want to monitor classification accuracy.

In [18]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['accuracy'])

Now we can create an instance of an ImageDataGenerator called train_datagen and a train_generator by calling train_datagen.flow_from_directory. ImageGenerator here is used to implicity label the training data based on the name of the subdirectory we created earlier. 

In [19]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory(
        '/tmp/h-or-s/',  # This is the source directory for training images
        target_size=(150, 150),  # All images will be resized to 150x150
        batch_size=10,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

# Expected output: 'Found 80 images belonging to 2 classes'

Found 80 images belonging to 2 classes.


Finally we can call model.fit and train for a number of epochs. We can pass in the previously defined callback method which will stop the training procedure when our monitored metric ('accuracy') has reached our desired goal, and stops unnecessary training of the model.

In [20]:
history = model.fit(
      train_generator,
      steps_per_epoch=8,  
      epochs=15,
      callbacks=[callbacks])
    
# Expected output: "Reached 99.9% accuracy so cancelling training!""

Epoch 1/15
8/8 [==============================] - 2s 244ms/step - loss: 1.4242 - accuracy: 0.5875
Epoch 2/15
8/8 [==============================] - 2s 241ms/step - loss: 0.5503 - accuracy: 0.7125
Epoch 3/15
8/8 [==============================] - 2s 241ms/step - loss: 0.1572 - accuracy: 0.9625
Epoch 4/15
8/8 [==============================] - ETA: 0s - loss: 0.0480 - accuracy: 1.0000
Reached 99.9% accuracy so cancelling training!
8/8 [==============================] - 2s 242ms/step - loss: 0.0480 - accuracy: 1.0000
